# Prompt for Agent

- Author: [Secludor](https://github.com/Secludor)
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

This notebook provides practical examples for building and evaluating dynamic AI agent prompts using LangChain. The notebook is organized around two main templates:

1. **AI_CAREER_PLANNER_AGENT_PROMPT**  
   This section demonstrates how to construct a robust prompt template that generates personalized study plans for users aspiring to become AI engineers. It covers:
   - **Tool Definitions**: Functions such as `resource_recommender` and `timeline_generator` are implemented to tailor learning resources and study timelines according to the user's skill gaps.
   - **Prompt Template**: The prompt template clearly defines the agent's role and behavioral guidelines, establishes working memory management, and integrates tools for generating actionable, structured study plans.
   - **Memory Initialization and Response Formatting**: It ensures that conversation history is maintained and outputs are formatted in a clear Markdown structure.
   - **Runnable Chain**: All components are integrated into a chain that processes user input, invokes the tools, and produces a final study plan, demonstrating flexibility and user-specific customization.

2. **PROMPT_QUALITY_EVALUATION_TEMPLATE**  
   This section illustrates a two-part process:
   - **Prompt Generation**: A generation template is employed to create prompts based on a user’s goal (e.g., developing a study plan for an AI engineering career).
   - **Prompt Evaluation**: An evaluation template is defined for a Prompt Quality Reviewer agent that verifies the generated prompt against defined criteria such as identity clarity, memory management, tool integration, plan completeness, and overall quality.  
     *Note*: Every literal `{` and `}` inside the JSON block is replaced with `{{` and `}}` to avoid unintended substitutions during Python’s formatting process.

Together, these templates showcase a modular and scalable approach to designing, generating, and evaluating dynamic prompts for AI agents. By integrating tool functions, context-aware memory management, and rigorous evaluation criteria, the notebook provides concrete examples of how to achieve intelligent task processing and high-quality agent output in real-world scenarios.

### Table of Contents
- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [PROMPT_QUALITY_EVALUATION_TEMPLATE](#prompt_quality_evaluation_template)
- [AI_CAREER_PLANNER_AGENT_PROMPT](#ai_career_planner_agent_prompt)

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install -U langchain-opentutorial


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

You can set API keys in a `.env` file or set them manually.

[Note] If you’re not using the `.env` file, no worries! Just enter the keys directly in the cell below, and you’re good to go.

In [3]:
from dotenv import load_dotenv
from langchain_opentutorial import set_env

# Attempt to load environment variables from a .env file; if unsuccessful, set them manually.
if not load_dotenv():
    set_env(
        {
            "OPENAI_API_KEY": "",
            "LANGCHAIN_API_KEY": "",
            "LANGCHAIN_TRACING_V2": "true",
            "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
            "TAVILY_API_KEY": "",
        }
    )

# set the project name same as the title
set_env(
    {
        "LANGCHAIN_PROJECT": "agentic-prompt",
    }
)

Environment variables have been set successfully.


## AI_CAREER_PLANNER_AGENT_PROMPT

This section demonstrates the implementation of the **AI_CAREER_PLANNER_AGENT_PROMPT**, which is designed to generate personalized study plans for users aiming to become AI engineers. Below is a detailed explanation of each part of the code:

### **1. Tool Definitions**
- **Purpose**: Two tools are defined to assist in creating the study plan:
  - **`resource_recommender`**: Recommends learning resources based on the user's current skills and target skills.
    - Input: User's current skills and desired target skills.
    - Output: A list of recommended resources, including course names, descriptions, and URLs.
  - **`timeline_generator`**: Generates a milestone-based study timeline.
    - Input: User's current skills and available study time (in weeks).
    - Output: A structured timeline with weekly goals.

- **Key Features**:
  - The tools ensure that the recommendations and timelines are tailored to the user's skill gaps.
  - Default target skills include Python, Linear Algebra, and Deep Learning, which are essential for AI engineering.

### **2. Prompt Template**
- **Purpose**: The `ChatPromptTemplate` defines the behavior and role of the AI agent. It provides clear instructions for generating a structured study plan.
- **Key Elements**:
  - **Agent Identity**: Specifies that the agent is an "AI Career Planning Specialist" with expertise in AI research and education.
  - **Behavioral Guidelines**:
    - Analyze skill gaps.
    - Recommend accessible learning resources.
    - Create realistic timelines with milestones.
  - **Working Memory Management**:
    - Tracks the current task context, progress, and pending actions.
    - Maintains awareness of conversation history and user input.
  - **Tool Integration**:
    - Utilizes `resource_recommender` for learning resources.
    - Uses `timeline_generator` for creating timelines.

- **Output Requirements**:
  - The response must be in Markdown format with sections for skill gap analysis, recommended resources, a timeline, and progress tracking checkpoints.

### **3. Memory Initialization**
- **Purpose**: Initializes memory using `ConversationKGMemory` to maintain conversation history and context across interactions.
- **Error Handling**:
  - If memory initialization fails, it falls back to a default empty memory state to ensure robustness.

### **4. Response Formatting**
- **Purpose**: Formats the output from the LLM or tool chain into a structured Markdown response for better readability.
- **Key Features**:
  - Combines recommendations and timeline into a single cohesive response.
  - Ensures that each section (resources, timeline) is clearly presented with actionable details.

### **5. Runnable Chain**
- **Purpose**: Combines all components into a runnable chain that processes user input and generates a personalized study plan.
- **Steps in the Chain**:
  1. Passes user input directly using `RunnablePassthrough`.
  2. Loads conversation history using `RunnableLambda(load_memory)`.
  3. Executes tools (`resource_recommender`, `timeline_generator`) to generate recommendations and timelines.
  4. Processes inputs through the prompt template (`prompt`) to guide the LLM's behavior.
  5. Formats the final output using `RunnableLambda(format_response)`.

### **6. Example Usage**
- Demonstrates how to use the runnable chain to generate a study plan for a user aiming to become an AI engineer.
- Example Input:
  ```python
  {
      "input": "I want to become an AI engineer. I know {current_skills}.",
      "current_skills": ["Basic Python"],
      "target_skills": ["Python", "Linear Algebra", "Deep Learning"],
      "weeks_available": 12,
  }
  ```
- Example Output (Formatted Markdown):
  ```markdown
  # AI Engineering Career Study Plan

  ## Recommended Resources
  - **Python**: [Python for Everybody (Coursera)](https://www.coursera.org/specializations/python)
  - **Linear Algebra**: [Khan Academy Linear Algebra](https://www.khanacademy.org/math/linear-algebra)
  - **Deep Learning**: [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning)

  ## Study Timeline
  - Week 1: Python Basics (2 weeks)
  - Week 3: Linear Algebra Fundamentals (2 weeks)
  - Week 5: Deep Learning Foundations (4 weeks)
  - Week 9: Practical Projects (4 weeks)

  ## Progress Tracking
  Weekly self-assessments and milestone reviews are recommended to ensure steady progress.
  ```

### Summary
The **AI_CAREER_PLANNER_AGENT_PROMPT** section showcases how to integrate tools, memory management, and prompt templates into a cohesive system for generating actionable study plans. By leveraging LangChain's modular components, this implementation ensures flexibility, scalability, and user-specific customization.

In [ ]:
from typing import Dict, List
from langchain_community.memory.kg import ConversationKGMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

# Initialize the LLM with the desired model and configuration.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# -----------------------------------------------------------
# Define Tools for Resource Recommendation and Timeline Generation
# -----------------------------------------------------------


def resource_recommender(input_data: Dict) -> Dict:
    """
    Recommend learning resources based on the user's current skills
    and target skills for an AI engineering career.

    Input: {
      "current_skills": List,
      "target_skills": List (default: ["Python", "Linear Algebra", "Deep Learning"])
    }
    Returns a dictionary with a list of recommendations.
    """
    current_skills = input_data.get("current_skills", [])
    target_skills = input_data.get(
        "target_skills", ["Python", "Linear Algebra", "Deep Learning"]
    )
    recommendations = []

    for skill in target_skills:
        if skill not in current_skills:
            if skill == "Python":
                recommendations.append(
                    {
                        "skill": skill,
                        "resource": "Python for Everybody (Coursera)",
                        "url": "https://www.coursera.org/specializations/python",
                    }
                )
            elif skill == "Linear Algebra":
                recommendations.append(
                    {
                        "skill": skill,
                        "resource": "Khan Academy Linear Algebra",
                        "url": "https://www.khanacademy.org/math/linear-algebra",
                    }
                )
            elif skill == "Deep Learning":
                recommendations.append(
                    {
                        "skill": skill,
                        "resource": "Deep Learning Specialization",
                        "url": "https://www.coursera.org/specializations/deep-learning",
                    }
                )
    return {"recommendations": recommendations}


def timeline_generator(input_data: Dict) -> Dict:
    """
    Generate a milestone-based study timeline based on user input.

    Input: {
      "current_skills": List,
      "weeks_available": Integer (default: 12)
    }
    Returns a dictionary containing a timeline.
    """
    current_skills = input_data.get("current_skills", [])
    weeks_available = input_data.get(
        "weeks_available", 12
    )  # It doesn't use in this example.
    timeline = []
    week = 1

    # Schedule Python basics if not mastered.
    if "Python" not in current_skills:
        timeline.append({"week": week, "goal": "Python Basics", "duration": "2 weeks"})
        week += 2

    # Continue with other core subjects.
    timeline.extend(
        [
            {
                "week": week,
                "goal": "Linear Algebra Fundamentals",
                "duration": "2 weeks",
            },
            {
                "week": week + 2,
                "goal": "Deep Learning Foundations",
                "duration": "4 weeks",
            },
            {"week": week + 6, "goal": "Practical Projects", "duration": "4 weeks"},
        ]
    )
    return {"timeline": timeline}


# -----------------------------------------------------------
# Define the Prompt Template for the AI Career Planner Agent.
# -----------------------------------------------------------

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a specialized agent with the following identity and operational parameters:

Identity:
- Role: AI Career Planning Specialist
- Domain: AI Research and Engineering Education
- Purpose: Create personalized AI career preparation plans
- Interaction Style: Structured, supportive, and detail-oriented

Behavioral Guidelines:
- Primary Goals:
  - Analyze skill gaps in AI/ML knowledge
  - Recommend targeted learning resources
  - Generate achievable study timelines
- Constraints:
  - Use only verified and accessible learning resources
  - Maintain realistic time expectations
  - Consider user's current skill level
- Success Criteria:
  - Clear progression path with measurable milestones
  - Resource recommendations matched to skill level
  - Achievable timeline based on user availability

Current Date: February 12, 2025

Working Memory Management:
- Current Task Context:
  - Active objective: Create personalized AI career study plan
  - Task progress: Analyzing user input and requirements
  - Pending actions: Resource matching and timeline creation
- Recent Interaction State:
  - Last user input: {input}
  - Conversation history: {history}
  - Current focus: Skill gap analysis and planning

Tool Integration:
- Resource Recommender: Match learning resources to identified skill gaps
- Timeline Generator: Create structured study schedule
- Progress Tracker: Monitor and adjust study milestones

Output Requirements:
1. Structured Markdown format with clear sections
2. Specific, actionable recommendations
3. Timeline with realistic milestones
4. Progress tracking checkpoints

Please maintain awareness of:
- User's current skill level
- Available study time
- Learning resource accessibility
- Skill dependencies and prerequisites
""",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)


# -----------------------------------------------------------
# Initialize and Integrate Memory with Error Handling
# -----------------------------------------------------------

try:
    memory = ConversationKGMemory(llm=llm)
except Exception as e:
    print(f"Memory initialization error: {e}")
    memory = None


def load_memory(input_dict: Dict) -> List:
    """
    Load conversation history from memory as a list of messages,
    ensuring the correct data format.
    """
    if memory is None:
        return []  # Return an empty history if memory is unavailable.
    try:
        history = memory.load_memory_variables(
            {"input": input_dict.get("input", "")}
        ).get("history", [])
        if not isinstance(history, list):
            history = []
        return history
    except Exception as e:
        print(f"Memory loading error: {e}")
        return []


# -----------------------------------------------------------
# Response Formatting Function
# -----------------------------------------------------------


def format_response(llm_response) -> str:
    """
    Format the LLM output (or a consolidated tool output dictionary)
    into a structured Markdown plan.
    """
    try:
        # If llm_response is an object with a 'content' attribute.
        if hasattr(llm_response, "content"):
            return llm_response.content
        if isinstance(llm_response, str):
            return llm_response
        if isinstance(llm_response, dict):
            recommendations = llm_response.get("recommendations", [])
            timeline = llm_response.get("timeline", [])
            response = "# AI Engineering Career Study Plan\n\n"
            response += "## Recommended Resources\n"
            for rec in recommendations:
                response += f"- **{rec['skill']}**: [{rec['resource']}]({rec['url']})\n"
            response += "\n## Study Timeline\n"
            for item in timeline:
                response += (
                    f"- Week {item['week']}: {item['goal']} ({item['duration']})\n"
                )
            return response
        return str(llm_response)
    except Exception as e:
        return f"Error formatting response: {str(e)}\nRaw response: {str(llm_response)}"


# -----------------------------------------------------------
# Build the Runnable Chain for the Agent
# -----------------------------------------------------------

chain = (
    {
        "input": RunnablePassthrough(),  # Pass the user input unchanged.
        "history": RunnableLambda(load_memory),  # Load conversation history.
        "recommendations": RunnableLambda(
            lambda x: resource_recommender(x)
        ),  # Get resource recommendations.
        "timeline": RunnableLambda(
            lambda x: timeline_generator(x)
        ),  # Generate study timeline.
    }
    | prompt  # Feed the consolidated inputs into the prompt.
    | llm  # Generate a refined response using the LLM.
    | RunnableLambda(format_response)  # Format the final output.
)

# -----------------------------------------------------------
# Example Usage: Generate a Study Plan for AI Research/Engineering Career Preparation
# -----------------------------------------------------------

if __name__ == "__main__":
    try:
        # Example user input with dynamic parameters indicated with {}.
        user_input = {
            "input": "I want to become an AI engineer. I know {current_skills}.",
            "current_skills": ["Basic Python"],
            "target_skills": ["Python", "Linear Algebra", "Deep Learning"],
            "weeks_available": 12,
        }

        # Execute the chain to produce a structured study plan.
        response = chain.invoke(user_input)
        print(response)

    except Exception as e:
        print(f"Error executing chain: {e}")

# AI Career Preparation Plan

## Overview
This plan is designed to help you transition from your current skill level to becoming an AI engineer. Based on your current skills and target skills, we will focus on enhancing your Python proficiency, building a strong foundation in Linear Algebra, and introducing you to Deep Learning concepts.

## Current Skills
- Basic Python

## Target Skills
1. Intermediate to Advanced Python
2. Linear Algebra
3. Deep Learning

## Study Timeline
You have 12 weeks available for study. The timeline below outlines a structured approach to achieve your target skills within this period.

### Weeks 1-4: Python Proficiency
**Objective:** Transition from basic to intermediate/advanced Python skills.

- **Resources:**
  - [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/) - Free online book and course
  - [Python for Everybody](https://www.coursera.org/specializations/python) - Coursera specialization (audit for free)
  - [LeetCode](https

In [6]:
from typing import Dict, List, Any
from langchain_community.memory.kg import ConversationKGMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

# Initialize the LLM with the desired configuration.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


# -----------------------------------------------------------
# Tool Definitions
# -----------------------------------------------------------
def resource_recommender(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Recommend learning resources based on the user's current and target skills
    for an AI engineering career.

    Expected Input:
      {
          "current_skills": List[str],
          "target_skills": List[str]  // Default: ["Python", "Linear Algebra", "Deep Learning"]
      }

    Returns:
      {
          "recommendations": List[{
              "skill": str,
              "resource": str,
              "url": str
          }]
      }
    """
    current_skills = input_data.get("current_skills", [])
    target_skills = input_data.get(
        "target_skills", ["Python", "Linear Algebra", "Deep Learning"]
    )
    recommendations = []

    for skill in target_skills:
        if skill not in current_skills:
            if skill == "Python":
                recommendations.append(
                    {
                        "skill": skill,
                        "resource": "Python for Everybody (Coursera)",
                        "url": "https://www.coursera.org/specializations/python",
                    }
                )
            elif skill == "Linear Algebra":
                recommendations.append(
                    {
                        "skill": skill,
                        "resource": "Khan Academy Linear Algebra",
                        "url": "https://www.khanacademy.org/math/linear-algebra",
                    }
                )
            elif skill == "Deep Learning":
                recommendations.append(
                    {
                        "skill": skill,
                        "resource": "Deep Learning Specialization (Coursera)",
                        "url": "https://www.coursera.org/specializations/deep-learning",
                    }
                )
    return {"recommendations": recommendations}


def timeline_generator(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Generate a milestone-based study timeline using user inputs.

    Expected Input:
      {
          "current_skills": List[str],
          "weeks_available": int  // Default: 12
      }

    Returns:
      {
          "timeline": List[{
              "week": int,
              "goal": str,
              "duration": str
          }]
      }
    """
    current_skills = input_data.get("current_skills", [])
    weeks_available = input_data.get("weeks_available", 12)
    timeline = []
    week = 1

    # Schedule Python Basics if not already mastered.
    if "Python" not in current_skills:
        timeline.append({"week": week, "goal": "Python Basics", "duration": "2 weeks"})
        week += 2

    # Define subsequent study milestones.
    timeline.extend(
        [
            {
                "week": week,
                "goal": "Linear Algebra Fundamentals",
                "duration": "2 weeks",
            },
            {
                "week": week + 2,
                "goal": "Deep Learning Foundations",
                "duration": "4 weeks",
            },
            {"week": week + 6, "goal": "Practical Projects", "duration": "4 weeks"},
        ]
    )
    return {"timeline": timeline}


# -----------------------------------------------------------
# Define the Unified Prompt Template
# -----------------------------------------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a specialized agent with the following identity and operational parameters:

Identity:
- Role: AI Career Planning Specialist
- Domain: AI Research and Engineering Education
- Purpose: Create personalized AI career preparation plans for users.
- Interaction Style: Structured, supportive, and detail-oriented.

Behavioral Guidelines:
- Primary Goals:
  - Analyze the user's current skill gaps in AI/ML.
  - Recommend targeted and accessible learning resources.
  - Generate a realistic study timeline with clear milestones.
- Constraints:
  - Use only verified and accessible resources.
  - Consider available study time and user’s current skill level.
- Success Criteria:
  - Provide a clear progression path with measurable milestones.
  - Deliver recommendations that align with the user's profile.

Working Memory Management:
- Current Task Context:
  - Active objective: "Create a personalized AI career study plan."
  - Task progress: "Analyzing user input and preparing recommendations."
  - Pending actions: "Match resources and generate timeline."
- Recent Interaction:
  - Last user input: {input}
  - Conversation history: {history}

Tool Integration:
- Resource Recommender: To identify learning resources based on skill gaps.
- Timeline Generator: To build a structured study schedule.

Output Requirements:
- Response must be in Markdown with the following sections:
  1. Skill Gap Analysis
  2. Recommended Resources (with URLs)
  3. Study Timeline with milestones
  4. Next steps and progress tracking

Current Date: February 12, 2025
Relevant Information:
{history}
""",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# -----------------------------------------------------------
# Memory Initialization with Robust Error Handling
# -----------------------------------------------------------
try:
    memory = ConversationKGMemory(llm=llm)
except Exception as e:
    print(f"Memory initialization error: {e}")
    memory = None


def load_memory(input_dict: Dict[str, Any]) -> List[Any]:
    """
    Load conversation history from memory ensuring it is returned as a list.
    Returns an empty list if the memory is unavailable or parsing fails.
    """
    if memory is None:
        return []
    try:
        history = memory.load_memory_variables(
            {"input": input_dict.get("input", "")}
        ).get("history", [])
        return history if isinstance(history, list) else []
    except Exception as e:
        print(f"Memory loading error: {e}")
        return []


# -----------------------------------------------------------
# Response Formatter Function
# -----------------------------------------------------------
def format_response(llm_response: Any) -> str:
    """
    Format the output from the language model or tool chain into a structured Markdown response.
    """
    try:
        if hasattr(llm_response, "content"):
            return llm_response.content
        if isinstance(llm_response, str):
            return llm_response
        if isinstance(llm_response, dict):
            recommendations = llm_response.get("recommendations", [])
            timeline = llm_response.get("timeline", [])

            md_response = "# AI Engineering Career Study Plan\n\n"
            md_response += "## Recommended Resources\n"
            for rec in recommendations:
                md_response += (
                    f"- **{rec['skill']}**: [{rec['resource']}]({rec['url']})\n"
                )

            md_response += "\n## Study Timeline\n"
            for item in timeline:
                md_response += (
                    f"- Week {item['week']}: {item['goal']} ({item['duration']})\n"
                )
            return md_response
        return str(llm_response)
    except Exception as e:
        return f"Error formatting response: {str(e)}\nRaw response: {str(llm_response)}"


# -----------------------------------------------------------
# Build the Runnable Chain
# -----------------------------------------------------------
chain = (
    {
        "input": RunnablePassthrough(),  # Directly pass the user input.
        "history": RunnableLambda(load_memory),  # Load conversation history.
        "recommendations": RunnableLambda(
            lambda x: resource_recommender(x)
        ),  # Execute resource recommender tool.
        "timeline": RunnableLambda(
            lambda x: timeline_generator(x)
        ),  # Execute timeline generator tool.
    }
    | prompt  # Feed consolidated input, history, and tool outputs to the prompt.
    | llm  # Generate the refined response using the LLM.
    | RunnableLambda(format_response)  # Format the final output into Markdown.
)

# -----------------------------------------------------------
# Example Usage: Generate a Study Plan for AI Research/Engineering Career Preparation
# -----------------------------------------------------------
if __name__ == "__main__":
    try:
        # Example user input with dynamic parameters indicated by placeholders.
        user_input = {
            "input": "I want to become an AI engineer. I know {current_skills}.",
            "current_skills": ["Basic Python"],
            "target_skills": ["Python", "Linear Algebra", "Deep Learning"],
            "weeks_available": 12,
        }

        # Execute the chain and print the formatted response.
        response = chain.invoke(user_input)
        print(response)

    except Exception as e:
        print(f"Error executing chain: {e}")

# Personalized AI Career Study Plan

## 1. Skill Gap Analysis

Based on your current skills and target skills, here is the analysis:

- **Current Skills**: Basic Python
- **Target Skills**: 
  - Advanced Python
  - Linear Algebra
  - Deep Learning

### Identified Skill Gaps:
1. **Advanced Python**: You need to enhance your Python skills to include data manipulation, libraries like NumPy and Pandas, and object-oriented programming.
2. **Linear Algebra**: Fundamental understanding of vectors, matrices, and operations is crucial for AI.
3. **Deep Learning**: Understanding neural networks, backpropagation, and frameworks like TensorFlow or PyTorch.

## 2. Recommended Resources

Here are some verified and accessible resources to help you fill these gaps:

### Advanced Python
- **Course**: [Python for Data Science and AI](https://www.coursera.org/learn/python-for-applied-data-science-ai) - Coursera
- **Book**: [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

### 

## PROMPT_QUALITY_EVALUATION_TEMPLATE

This template is designed to evaluate the quality of generated prompts produced for AI agent tasks. It follows a two-part structure:

1. **Prompt Generation:** Creates a high-quality prompt based on a user-defined goal.
2. **Prompt Evaluation:** Uses an evaluation template to review the generated prompt against key criteria and returns structured feedback.

A key note for this template is that every literal `{` and `}` inside the JSON block has been replaced with `{{` and `}}` to ensure that Python’s `.format()` method does not mistakenly treat them as placeholders.

---

### Part 1: Prompt Generation

- **Objective:**  
  A prompt is generated for the purpose of creating a study plan for becoming an AI engineer. This generated prompt is meant to guide an AI agent in creating a comprehensive career preparation plan.

- **Implementation Details:**
  - A **generation template** is created using `ChatPromptTemplate.from_messages()`.  
  - The system message instructs the agent (acting as a Prompt Generator) on the required components of the prompt:
    - Agent Identity and Role
    - Behavioral Guidelines and Success Criteria
    - Task Description and Input Parameters
    - Instructions for memory management and tool integration
  - A human message accepts the `{user_goal}` as input.
  - The `user_goal_text` variable contains an example goal where the user specifies they want to develop a study plan targeting improvements in Python programming, data analysis, and fundamental machine learning concepts.
  - The prompt is then generated by calling `generation_template.format_messages(user_goal=user_goal_text)` and concatenated into a single string (`generated_prompt_str`).

---

### Part 2: Prompt Evaluation

- **Objective:**  
  The evaluation template reviews the generated prompt to ensure that it adheres to best practices for prompt quality. It checks if various aspects of the prompt are well defined and actionable.

- **Implementation Details:**
  - An **evaluation template** is defined using `ChatPromptTemplate.from_messages()` with a system message that:
    - Sets the task by defining the role of a **Prompt Quality Reviewer**.
    - Provides the **Content to Review** using a `{generated_prompt}` placeholder.
    - Outlines **Verification Steps** that cover:
      1. **Identity and Role Clarity:** Checks if the agent’s role, purpose, and behavioral guidelines are clear.
      2. **Memory Management:** Assesses the structure for managing context.
      3. **Tool Integration:** Reviews the mention and use of necessary tools.
      4. **Plan Completeness:** Ensures all aspects required for the task are present, including clear input parameters.
      5. **Quality Standards:** Evaluates clarity, conciseness, and overall effectiveness.
    - Specifies a **Return Format** in JSON. Notice the use of double curly braces (`{{` and `}}`) in the JSON block. This prevents Python’s string formatter from interpreting them as variable placeholders. It outlines expected keys like `"score"`, `"feedback"` (with sub-fields like `identity_clarity`, `memory_management`, etc.), and an array of `"improvement_suggestions"`.
    - Provides an **Example Response** within the template to illustrate how the evaluation should be formatted.
  - A human message passes the `{generated_prompt}` into the evaluation template.
  - The code then calls `evaluation_template.format_messages(generated_prompt=generated_prompt_str)` to properly inject the generated prompt into the evaluation template.
  - Finally, the evaluation prompt is sent to the LLM (`ChatOpenAI`) that returns a structured evaluation response based on the defined criteria.

---

### How It Works Together

1. **Generation:**  
   The user goal is processed to create a detailed prompt that instructs an AI agent on how to build a personalized study plan. The strategy is to ensure that all necessary details (like identity, behavioral guidelines, and instructions for memory and tool integration) are clearly defined.

2. **Evaluation:**  
   Once generated, the prompt is reviewed by another sub-agent (acting as a Prompt Quality Reviewer). The evaluation checks for clarity, completeness, and technical feasibility based on the provided criteria.

3. **JSON Formatting:**  
   To avoid conflict with Python’s string formatting, all literal curly braces in the JSON block are doubled. This ensures that the evaluation response is formatted as intended.

---

This practical example demonstrates a modular approach where one agent generates a prompt while another evaluates its quality. Both parts integrate with LangChain's templating system, allowing for the creation of dynamic, high-quality prompts that can adapt based on user input.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the LLM with the desired configuration.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# -----------------------------------------------------------
# Part 1: Prompt Generation
# -----------------------------------------------------------
# Define the generation prompt template with a system message and a human message.
generation_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a Prompt Generator specialized in crafting effective prompts for AI agents.
Your task is to create a clear and structured prompt based on the user's goal.
Ensure the generated prompt includes the following elements:
- Agent Identity and Role
- Behavioral Guidelines and Success Criteria
- Task Description and Input Parameters
- Any necessary instructions for memory management and tool integration
""",
        ),
        ("human", "{user_goal}"),
    ]
)

# Example user goal describing the purpose.
user_goal_text = (
    "I want to develop a study plan for becoming an AI engineer. "
    "My focus areas are improving my Python programming, learning data analysis, "
    "and understanding machine learning fundamentals. "
    "Please generate a prompt that guides an AI agent to create a comprehensive career preparation plan."
)

# Generate the prompt from the template.
generated_prompt_messages = generation_template.format_messages(
    user_goal=user_goal_text
)
# Join the message contents into a single string (avoiding the 'list' object error).
generated_prompt_str = "\n".join(msg.content for msg in generated_prompt_messages)
print("Generated Prompt:\n", generated_prompt_str)

# -----------------------------------------------------------
# Part 2: Prompt Evaluation
# -----------------------------------------------------------
# Define the evaluation template.
evaluation_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """### Task
You are a Prompt Quality Reviewer tasked with verifying the quality of agent prompts.

### Content to Review
{generated_prompt}

### Verification Steps
1. Identity and Role Clarity  
   - Is the agent's role and purpose clearly defined?  
   - Are behavioral guidelines specific and actionable?

2. Memory Management  
   - Is the prompt structured to track context and manage memory effectively?

3. Tool Integration  
   - Are any necessary tools and their usages mentioned clearly?

4. Plan Completeness  
   - Does the prompt cover all essential aspects required for the intended task?  
   - Is there clear guidance on expected input parameters?

5. Quality Standards  
   - Is the prompt clear, concise, and effective?

### Return Format
Provide your evaluation in JSON format as follows:

```
{{
  "score": <integer_between_0_and_100>,
  "feedback": {{
    "identity_clarity": "<feedback>",
    "memory_management": "<feedback>",
    "tool_integration": "<feedback>",
    "plan_completeness": "<feedback>",
    "quality_standards": "<feedback>"
  }},
  "improvement_suggestions": [
    "<suggestion_1>",
    "<suggestion_2>",
    "<suggestion_3>"
  ]
}}
```

### Example Response
```
{{
  "score": 85,
  "feedback": {{
    "identity_clarity": "Agent role and behavioral guidelines are well defined, but success criteria could be more specific.",
    "memory_management": "The working memory structure is comprehensive but lacks clear cleanup protocols.",
    "tool_integration": "Tool selection is robust; however, error handling details could be improved.",
    "plan_completeness": "All essential elements are present, but additional details on input parameters are needed.",
    "quality_standards": "The prompt is generally clear, though conciseness can be improved."
  }},
  "improvement_suggestions": [
    "Add specific memory cleanup triggers.",
    "Include detailed error recovery procedures.",
    "Specify maximum retry attempts for failed operations."
  ]
}}
```
""",
        ),
        ("human", "{generated_prompt}"),
    ]
)

# Format the evaluation prompt with the generated prompt inserted.
evaluation_messages = evaluation_template.format_messages(
    generated_prompt=generated_prompt_str
)

# Invoke the LLM to evaluate the generated prompt.
evaluation_response = llm.invoke(evaluation_messages)
print("\nEvaluation Response:\n", evaluation_response.content)

Generated Prompt:
 You are a Prompt Generator specialized in crafting effective prompts for AI agents.
Your task is to create a clear and structured prompt based on the user's goal.
Ensure the generated prompt includes the following elements:
- Agent Identity and Role
- Behavioral Guidelines and Success Criteria
- Task Description and Input Parameters
- Any necessary instructions for memory management and tool integration

I want to develop a study plan for becoming an AI engineer. My focus areas are improving my Python programming, learning data analysis, and understanding machine learning fundamentals. Please generate a prompt that guides an AI agent to create a comprehensive career preparation plan.

Evaluation Response:
 {
  "score": 78,
  "feedback": {
    "identity_clarity": "The agent's role as a career preparation planner is clear, but the behavioral guidelines could be more detailed.",
    "memory_management": "The prompt does not explicitly address memory management, which co